# Pre-Process Decagon Data and Save Using Pickle

## Highlight



The frequency distribution of DD edge types is shown in the following figure. In Zinik(2018)'s work, they remove the DD edge types that each occurred in less than 500 drug combinations. Therefore, they use the sub-dataset of what we described and analyzed in this notebook.
![](hist_dd_edge.png)

Here is the summary of the data they use:
- protein: 19k
- protein edge: 1431k
- drug: 645
- drug edge: 6966k
- drug edge type: 963
- drug with protein link (dwp): 284
- dwp edge: 1810k
- drug feature: dim 10k, {0, 1} binary valued


## 1. Analysis Nodes, Edges, Node Features and Edge Types

In [ ]:
import csv


In [ ]:
# set data paths
pp_f = "../data/bio-decagon-ppi.csv"
dd_f = "../data/bio-decagon-combo.csv"
dp_f = "../data/bio-decagon-targets.csv"
ds_f = "../data/bio-decagon-mono.csv"


In [ ]:
# original data collection holder
p_set = set()                 # protein nodes
d_set = set()                 # drug nodes
combo_set = set()             # edge types
mono_set = set()              # additional drug features

# read file to list
pp_list, ddt_list, dp_list, ds_list = [], [], [], []   # lists of tuples

# set temporary variable
a, b, c = 0, 0, 0

### 1.1 protein-protein association network

Includes:

- physical interaction
- functional associations

Notation:

- node: protein
- edge: association or interaction

Number:

- 19081 nodes (min: 1; max: 110255169)
- 715612 edges

In [ ]:
with open(pp_f, 'r') as f:
    ppi = csv.reader(f)
    next(ppi)
    for [g1, g2] in ppi:
        a, b = int(g1), int(g2)
        p_set.add(a)
        p_set.add(b)
        pp_list.append((a, b))
print("{:d} protein nodes indexed from {:d} to {:d}.".format(len(p_set), min(p_set), max(p_set)))
print("{:d} protein-protein edges".format(len(pp_list)))

### 1.2 drug-drug association network with side effects

!! TIME COSTY SECTION

Notation:

- node: drug
- edge with typies: type of side effects that are associated with drug pairs

Number:

- 645 nodes
- 63473 edges
- 1317 edge types

In [ ]:
with open("../data/bio-decagon-combo.csv", "r") as f:
    ppi = csv.reader(f)
    next(ppi)
    for [d1, d2, t, n] in ppi:
        a, b, c = int(t.split('C')[-1]), int(d1.split('D')[-1]), int(d2.split('D')[-1])
        combo_set.add(a)
        d_set.add(b)
        d_set.add(c)
        ddt_list.append((b, c, a))
print("{:d} drug nodes indexed from {:d} to {:d}".format(len(d_set), min(d_set), max(d_set)))
print("{:d} drug-drug edges with {:d} edge types indexed from {:d} to {:d}"
      .format(len(ddt_list), len(combo_set), min(combo_set), max(combo_set)))


### 1.3 drug-protein association network

Notation:

- node: drug and genes(encode protein)
- edge: gene is targeted by drug

Number:

- 3932 nodes: 284 drug; 3648 gene
- 18690 edges

In [ ]:
p_temp_set = set()
d_temp_set = set()

with open("../data/bio-decagon-targets.csv", "r") as f:
    ppi = csv.reader(f)
    next(ppi)
    for [d, p] in ppi:
        a, b = int(d.split('D')[-1]), int(p)
        d_set.add(a)
        d_temp_set.add(a)
        p_set.add(b)
        p_temp_set.add(b)
        dp_list.append((a, b))
print("{:d} drug-protein edges".format(len(dp_list)))
print("{:d} proteins not in the ppi network".format(p_set.__len__() - 19081))
print("Protein numbers of {:d} indexed from {:d} to {:d}".format(len(p_temp_set), min(p_temp_set), max(p_temp_set)))
print("Drug numbers of {:d} indexed from {:d} to {:d}".format(len(d_temp_set), min(d_temp_set), max(d_temp_set)))

print("{:d} drugs not in the drug-drug network".format(d_set.__len__() - 645))


### 1.4 single drug side effects

Notation:

- node: drug and side-effect
- edge: side effect caused by drug

Number:

- 10825 nodes: 640 drug; 10184 side effect
- 174978 edges

In [ ]:
temp = d_set.__len__()
s_temp_set = set()
d_temp_set = set()
ds_list = []

with open("../data/bio-decagon-mono.csv", "r") as f:
    ppi = csv.reader(f)
    next(ppi)
    for [d, e, n] in ppi:
        a, b = int(e.split('C')[-1]), int(d.split('D')[-1])
        mono_set.add(a)
        d_set.add(b)
        s_temp_set.add(a)
        d_temp_set.add(b)
        ds_list.append((b, a))
print("{:d} drugs not in the drug-drug network."
      .format(d_set.__len__() - temp))
print("{:d} single drug side effects in drug pair side effects set."
      .format((combo_set & mono_set).__len__()))

print("Drug: {:d} indexed from {:d} to {:d}"
      .format(len(d_temp_set), min(d_temp_set), max(d_temp_set)))
print("Side effect: {:d} indexed from {:d} to {:d}"
      .format(len(s_temp_set), min(s_temp_set), max(s_temp_set)))
print("Edge: {:d}".format(len(ds_list)))

In [ ]:
# the numbers
num_gene = p_set.__len__()
num_drug = d_set.__len__()
num_edge_type = combo_set.__len__()
num_drug_additional_feature = mono_set.__len__()

print("Summary: ")
print(" -> Protein Node              : {:d}".format(num_gene))
print(" -> Drug    Node              : {:d}".format(num_drug))
print(" -> Drug    Pair  Side  Effect: {:d}".format(num_edge_type))
print(" -> Single  Drug  Side  Effect: {:d}".format(num_drug_additional_feature))

## 2. Index Drug Nodes, Protein Nodes, Drug Additional Features and Drug-Drug Edge Types

Note: Using gene presents its encode protein

In [ ]:
import scipy.sparse as sp

### 2.1 Re-indexing Stragy

xxx\_to\_new is a one-row sparse matrix:

- index: real drug/protein/side effect index  
- value: its new index 

xxx\_to\_old is a list:

- index: the re-allocated index
- value: original index


In [ ]:
# protein
gene_to_old = list(p_set)
gene_to_new = sp.csr_matrix((range(num_gene), ([0] * num_gene, gene_to_old)))

# drug
drug_to_old = list(d_set)
drug_to_new = sp.csr_matrix((range(num_drug), ([0] * num_drug, drug_to_old)))

# drug pair side effect
edge_type_to_old = list(combo_set)
edge_type_to_new = sp.csr_matrix((range(num_edge_type), ([0] * num_edge_type, edge_type_to_old)))

# single drug side effect
side_effect_to_old = list(mono_set)
side_effect_to_new = sp.csr_matrix((range(num_drug_additional_feature), ([0] * num_drug_additional_feature, side_effect_to_old)))


### 2.2 report the top 10 best and worst performance side effects

In [ ]:
# side effect name - original index
best_name = ["Mumps", "Carbuncle", "Coccydynia", "Tympanic membrane perfor", "Dyshidrosis", 
             "Spondylosis", "Schizoaffective disorder", "Breast dysplasia", "Ganglion", "Uterine polyp"
            ]
worst_name = ["Bleeding", "Body temperature increased",  "Emesis", "Renal disorder", "Leucopenia", 
              "Diarrhea", "Icterus", "Nausea", "Itch", "Anaemia"]
best_org_ind = [26780, 7078, 9193, 206504, 32633, 
                38019, 36337, 16034, 1258666, 156369] 
worst_org_ind = [19080, 15967, 42963, 22658, 23530, 
                 11991, 22346, 27497, 33774, 2871]

best_ind = [edge_type_to_new[0, i] for i in best_org_ind]
worst_ind = [edge_type_to_new[0, i] for i in worst_org_ind]

print(best_ind, worst_ind)


In [ ]:
edge_type_to_new[0, 26780]

## 3. Construct Sparse Adjacency Matrix

### 3.1 gene-gene adjacency matrix



In [ ]:
import numpy as np

In [ ]:
# set temporary variable
r, c, array_length = [], [], len(pp_list)

In [ ]:
for i in range(array_length):
    r.append(gene_to_new[0, pp_list[i][0]])
    c.append(gene_to_new[0, pp_list[i][1]])
    
gene_adj = sp.csr_matrix(([1] * array_length, (r, c)), shape=(num_gene, num_gene))

gene_degrees = np.array(gene_adj.sum(axis=0)).squeeze()


### 3.2 drug-gene & gene-drug adjacency matrix


In [ ]:
# set temporary variable
r, c, array_length = [], [], len(dp_list)

In [ ]:
for i in range(array_length):
    r.append(drug_to_new[0, dp_list[i][0]])
    c.append(gene_to_new[0, dp_list[i][1]])

drug_gene_adj = sp.csr_matrix(([1] * array_length, (r, c)), shape=(num_drug, num_gene))
gene_drug_adj = drug_gene_adj.transpose(copy=True)
    

### 3.3 drug-drug adjacency matrix list 

Note: this list is indexed by drug-drug edge type index

!! TIME COSTY SECTION

In [ ]:
# set temporary variable
# r: key is the index of edge type, value is a list of two lists [drug list, drug list]
r = {}        
array_length = len(ddt_list)


In [ ]:
# build drug-drug network by edge types
for i in range(array_length):
    c = edge_type_to_new[0, ddt_list[i][2]]
    if c not in r:
        r[c] = [drug_to_new[0, ddt_list[i][0]]], [drug_to_new[0, ddt_list[i][1]]]
    else:
        r[c][0].append(drug_to_new[0, ddt_list[i][0]])
        r[c][1].append(drug_to_new[0, ddt_list[i][1]])

# build adjacency matrix
drug_drug_adj_list = []
for i in range(num_edge_type):
    drug_drug_adj_list.append(sp.csr_matrix(([1] * len(r[i][0]), (r[i][0], r[i][1])), shape=(num_drug, num_drug)))

drug_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in drug_drug_adj_list]

                

In [ ]:
drug_drug_adj_list = []

for i in range(num_edge_type):
    drug_drug_adj_list.append(sp.load_npz("".join(["../data/drug-sparse-adj/type_", str(i), ".npz"])))
    

## 4. Generate Feature Vector

### 4.1 gene one-hot vector

In [ ]:
from utility.preprocessing import sparse_to_tuple

In [ ]:
gene_feat = sp.identity(num_gene)

gene_nonzero_feat, gene_num_feat = gene_feat.shape
gene_feat = sparse_to_tuple(gene_feat.tocoo())

### 4.2 drug feature vector with additional feature

In [ ]:
# index feature
r, c = list(range(num_drug)), list(range(num_drug))

# additional feature append to index feature
for (a, b) in ds_list:
    r.append(drug_to_new[0, a])
    c.append(side_effect_to_new[0, b] + num_drug)

array_length = num_drug + len(ds_list)
drug_feat_sparse = sp.csr_matrix(([1] * array_length, (r, c)), shape=(num_drug, num_drug + num_drug_additional_feature))

drug_nonzero_feat, drug_num_feat = drug_feat_sparse.shape[1], np.count_nonzero(drug_feat_sparse.sum(axis=0))
drug_feat = sparse_to_tuple(drug_feat_sparse.tocoo())


### 4.3 Visualization of side effects

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# dimension reduction
drug_embedded = TSNE(n_components=2).fit_transform(drug_feat_sparse.toarray())


In [ ]:
# random selecting five side effects for visualization
np.random.seed(19680801)
num_show_edge_index = 2
num_show_edge = 60

selected_edge_types = np.random.choice(num_edge_type, num_show_edge_index, replace=False)
selected_edge_types.tolist()

In [ ]:
plt.figure(figsize=(12, 12), dpi= 80, facecolor='w', edgecolor='k')
title = "Drug Embedding Before Training"
col = ['r', 'g', 'b', 'k', 'c', 'm', 'y']

plt.scatter(drug_embedded[:, 0], drug_embedded[:, 1], s=6, label='drug')
for i in range(num_show_edge_index):
    edge = selected_edge_types[i]
    r, c = drug_drug_adj_list[edge].nonzero()
    
    if len(r) > num_show_edge:
        selected_drug_pair_index = np.random.choice(len(r), num_show_edge, replace=False).tolist()
    else:
        selected_drug_pair_index = list(range(len(r)))
        
    for j in selected_drug_pair_index:
        [x1, y1] = drug_embedded[r[j], :]
        [x2, y2] = drug_embedded[c[j], :]
        
        plt.plot([x1, x2], [y1, y2], color=col[i], alpha=0.3, marker='*')
    

# setting
plt.title(title)
plt.grid()
plt.savefig("Drug Embedding before training2.pdf")
plt.show()

## 5. Save to Files

### 5.1 save graph infomation - redundant

In [ ]:
import pickle


In [ ]:
# index saver
with open('./data_decagon/edge_type_to_old.pkl', 'wb') as f:
    pickle.dump(edge_type_to_old, f)
with open('./data_decagon/drug_to_old', 'wb') as f:
    pickle.dump(drug_to_old, f)


In [ ]:
with open('data_decagon/graph_num_info.pkl', 'wb') as f:
    pickle.dump([num_gene, num_drug, num_edge_type, num_drug_additional_feature], f)



### 5.2 save sparse matrix - redundant

In [ ]:
# gene-gene adj
sp.save_npz("data_decagon/gene-sparse-adj.npz", gene_adj)

# gene-drug adj
sp.save_npz("data_decagon/drug-gene-sparse-adj.npz", drug_gene_adj)
sp.save_npz("data_decagon/gene-drug-sparse-adj.npz", gene_drug_adj)

# drug-drug adj
for i in range(num_edge_type):
    sp.save_npz("".join(["data_decagon/drug-sparse-adj/type_", str(i), ".npz"]), drug_drug_adj_list[i])
    

### 5.3 save drug sparse feature vectors - redundant

In [ ]:
# drug feature vector
sp.save_npz("data_decagon/drug-feature-sparse.npz", drug_feat_sparse)


### 6.0 build full drug graph

In [ ]:
import threading

In [ ]:
class AtomicInteger():
    def __init__(self, value=0):
        self._value = int(value)
        self._lock = threading.Lock()
        
    def inc(self, d=1):
        with self._lock:
            self._value += int(d)
            return self._value

    def dec(self, d=1):
        return self.inc(-d)    

    @property
    def value(self):
        with self._lock:
            return self._value

    @value.setter
    def value(self, v):
        with self._lock:
            self._value = int(v)
            return self._value

atomic_int=AtomicInteger()

In [ ]:
import networkx as nx

gene_net = nx.from_numpy_matrix(gene_adj.todense())
# wir muessen hier noch noch den nodes sagen, dass sie gene sind

In [ ]:
net_list = []
atomic_int=AtomicInteger()

for i in range(len(drug_drug_adj_list)):
    temp_net = nx.from_numpy_matrix(drug_drug_adj_list[i].todense())
    if temp_net.size()>=500:
        G=nx.MultiGraph()
        G.add_nodes_from(temp_net)
        for edge in list(temp_net.edges):
            G.add_edge(edge[0],edge[1],label="test_" + str(i),key=atomic_int.inc())
        net_list.append(G)

super time costly !!!

In [ ]:
full_drug_graph = nx.compose_all(net_list)
#nx.write_gpickle(full_drug_graph,"full_drug_graph.pkl")

better:

In [ ]:
import networkx as nx
full_drug_graph=nx.read_gpickle("full_drug_graph.pkl")

In [ ]:
full_drug_graph.size()

In [ ]:
type(full_drug_graph)

In [ ]:
from stellargraph import StellarGraph

drug_graph = StellarGraph.from_networkx(full_drug_graph)

In [ ]:
print(drug_graph.info())

### 7.0 Link prediction

In [ ]:
epochs = 300
embedding_dimension = 100
negative_samples = 2000

In [ ]:
import pandas as pd


df = pd.DataFrame(list(drug_graph.edges(include_edge_type=True)))
df.columns = ['source','target', 'label']
df = df[['source', 'label', 'target']]
print(df)

In [ ]:
#Counter(df.label)

In [ ]:
from sklearn.model_selection import train_test_split
drug_train, drug_temp = train_test_split(df, test_size=0.3, random_state=42,stratify=df.label)

drug_test, drug_valid = train_test_split(drug_temp, test_size=0.5, random_state=42,stratify=drug_temp.label)

drug_train.reset_index(drop=True, inplace=True)
drug_test.reset_index(drop=True, inplace=True)
drug_valid.reset_index(drop=True, inplace=True)

In [ ]:
from stellargraph import datasets, utils
from tensorflow.keras import callbacks, optimizers, losses, metrics, regularizers, Model
from stellargraph.mapper import KGTripleGenerator
from stellargraph.layer import DistMult


# Train model
drug_gen = KGTripleGenerator(
    drug_graph, batch_size=len(drug_train) // 10  # ~10 batches per epoch
)

drug_distmult = DistMult(
    drug_gen,
    embedding_dimension=embedding_dimension,
    embeddings_regularizer=regularizers.l2(1e-7),
)

drug_inp, drug_out = drug_distmult.in_out_tensors()

drug_model = Model(inputs=drug_inp, outputs=drug_out)

drug_model.compile(
    optimizer=optimizers.Adam(lr=0.001),
    loss=losses.BinaryCrossentropy(from_logits=True),
    metrics=[metrics.BinaryAccuracy(threshold=0.0)],
)

In [ ]:
drug_train_gen = drug_gen.flow(drug_train, negative_samples=negative_samples, shuffle=True)
drug_valid_gen = drug_gen.flow(drug_valid, negative_samples=negative_samples)

In [ ]:
#computative intensive!!!

drug_es = callbacks.EarlyStopping(monitor="val_loss", patience=50)
drug_history = drug_model.fit(
    drug_train_gen,
    validation_data=drug_valid_gen,
    epochs=epochs,
    callbacks=[drug_es],
    verbose=1,
)

In [ ]:
utils.plot_history(drug_history)

In [ ]:
#evalulate model
drug_smaller_gen = KGTripleGenerator(drug_graph, batch_size=5000)

drug_raw_ranks, drug_filtered_ranks = drug_distmult.rank_edges_against_all_nodes(drug_smaller_gen.flow(drug_test), drug_graph)

In [ ]:
# helper function to compute metrics from an array of ranks
def results_as_dataframe(mrr, hits_at_10, hits_at_1):
    return pd.DataFrame(
        [(mrr, hits_at_10, hits_at_1)], columns=["mrr", "hits at 10", "hits at 1"], index=["filtered"],
    )


def summarise(ranks):
    return results_as_dataframe(np.mean(1 / ranks), np.mean(ranks <= 10), np.mean(ranks <= 1))

In [ ]:
summarise(drug_filtered_ranks)

In [ ]:
0.034777	0.072666	0.007877

### additional embeddings

In [ ]:
#TODO: https://stellargraph.readthedocs.io/en/stable/api.html#knowledge-graph-models

from stellargraph.layer import ComplEx

# Train model
drug_gen = KGTripleGenerator(
    drug_graph, batch_size=len(drug_train) // 10  # ~10 batches per epoch
)

drug_ComplEx = ComplEx(
    drug_gen,
    embedding_dimension=embedding_dimension,
    embeddings_regularizer=regularizers.l2(1e-7),
)

drug_inp, drug_out = drug_ComplEx.in_out_tensors()

drug_model = Model(inputs=drug_inp, outputs=drug_out)

drug_model.compile(
    optimizer=optimizers.Adam(lr=0.0001),
    loss=losses.BinaryCrossentropy(from_logits=True),
    metrics=[metrics.BinaryAccuracy(threshold=0.0)],
)

In [ ]:
drug_train_gen = drug_gen.flow(drug_train, negative_samples=negative_samples, shuffle=True)
drug_valid_gen = drug_gen.flow(drug_valid, negative_samples=negative_samples)

drug_es = callbacks.EarlyStopping(monitor="val_loss", patience=50)
drug_history = drug_model.fit(
    drug_train_gen,
    validation_data=drug_valid_gen,
    epochs=epochs,
    callbacks=[drug_es],
    verbose=1,
)

In [ ]:
utils.plot_history(drug_history)

In [ ]:
from stellargraph.layer import RotatE

# Train model
drug_gen = KGTripleGenerator(
    drug_graph, batch_size=len(drug_train) // 10  # ~10 batches per epoch
)

drug_RotatE = RotatE(
    drug_gen,
    embedding_dimension=embedding_dimension,
    embeddings_regularizer=regularizers.l2(1e-7),
)

drug_inp, drug_out = drug_RotatE.in_out_tensors()

drug_model = Model(inputs=drug_inp, outputs=drug_out)

drug_model.compile(
    optimizer=optimizers.Adam(lr=0.001),
    loss=losses.BinaryCrossentropy(from_logits=True),
    metrics=[metrics.BinaryAccuracy(threshold=0.0)],
)

In [ ]:
drug_train_gen = drug_gen.flow(drug_train, negative_samples=negative_samples, shuffle=True)
drug_valid_gen = drug_gen.flow(drug_valid, negative_samples=negative_samples)

drug_es = callbacks.EarlyStopping(monitor="val_loss", patience=50)
drug_history = drug_model.fit(
    drug_train_gen,
    validation_data=drug_valid_gen,
    epochs=epochs,
    callbacks=[drug_es],
    verbose=1,
)

In [ ]:
utils.plot_history(drug_history)

In [ ]:
#evalulate model
drug_smaller_gen = KGTripleGenerator(drug_graph, batch_size=5000)

drug_raw_ranks, drug_filtered_ranks = drug_RotatE.rank_edges_against_all_nodes(drug_smaller_gen.flow(drug_test), drug_graph)

summarise(drug_filtered_ranks)

In [ ]:
from stellargraph.layer import Node2Vec

In [ ]:
from stellargraph.layer import Node2Vec
from stellargraph.mapper import Node2VecLinkGenerator, Node2VecNodeGenerator

batch_size=8

# Train model
drug_gen = KGTripleGenerator(
    drug_graph, batch_size=len(drug_train) // 10  # ~10 batches per epoch
)

# Define a Node2Vec training generator, which generates batches of training pairs
generator = Node2VecLinkGenerator(drug_graph, batch_size)

# Create the Node2Vec model
node2vec = Node2Vec(embedding_dimension, generator=generator)

drug_inp, drug_out = node2vec.in_out_tensors()

drug_model = Model(inputs=drug_inp, outputs=drug_out)

drug_model.compile(
    optimizer=optimizers.Adam(lr=0.001),
    loss=losses.BinaryCrossentropy(from_logits=True),
    metrics=[metrics.BinaryAccuracy(threshold=0.0)],
)

In [ ]:
drug_train_gen = drug_gen.flow(drug_train, negative_samples=negative_samples, shuffle=True)
drug_valid_gen = drug_gen.flow(drug_valid, negative_samples=negative_samples)

#drug_es = callbacks.EarlyStopping(monitor="val_loss", patience=50)
#drug_history = drug_model.fit(
#    drug_train_gen,
#    validation_data=drug_valid_gen,
#    epochs=epochs,
#    callbacks=[drug_es],
#    verbose=1,
#)

In [ ]:
utils.plot_history(drug_history)